In [2]:
%load_ext autoreload
%autoreload 2

from threed import ThreeD
import h5py

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [40]:
LOCAL_DATA = '/home/d/data/toufiq/segmentation.h5'
CLUSTER_DATA = '/n/coxfs01/leek/results/ECS_iarpa_20u_cube/segmentation.h5'
CLUSTER_STL_OUTPUT = '/n/coxfs01/haehn/STLs/'
TILEWIDTH = 1024

In [20]:
with h5py.File(LOCAL_DATA, 'r') as f:
    shape = f[f.keys()[0]].shape

In [38]:
# assume X,Y equal right now
tiles_Y = range(int(round(shape[1] / float(TILEWIDTH))))
tiles_X = range(int(round(shape[2] / float(TILEWIDTH))))

In [29]:
# we assume the following memory
bytes = TILEWIDTH * TILEWIDTH * shape[0] * 8
print 'MB', bytes / 1000000
# and we take 4 times as large just to be safe
memory = 4 * bytes / 1000000

MB 5872


In [36]:
# SLURM TEMPLATE
t = '''#!/bin/bash
#
# add all other SBATCH directives here...
#
#SBATCH -p cox
#SBATCH -n 1 # Number of cores
#SBATCH -N 1 # Ensure that all cores are on one machine #SBITCH --gres=gpu
#SBATCH --mem={MEMORY}
#SBATCH -t 10-12:00
#SBATCH --mail-type=ALL
#SBATCH --mail-user=haehn@seas.harvard.edu
#SBATCH -o /n/home05/haehn/SLURM/out-mesh-{Y}-{X}.txt
#SBATCH -e /n/home05/haehn/SLURM/err-mesh-{Y}-{X}.txt

source new-modules.sh
module load Anaconda/2.5.0-fasrc01
module load gcc/4.9.0-fasrc01

# custom HDF5 lib
export LIBRARY_PATH=/n/home05/haehn/nolearncox/src/hdf5-1.8.17/hdf5/lib:$LIBRARY_PATH
export LD_LIBRARY_PATH=/n/home05/haehn/nolearncox/src/hdf5-1.8.17/hdf5/lib:$LD_LIBRARY_PATH
export CPATH=/n/home05/haehn/nolearncox/src/hdf5-1.8.17/hdf5/include:$CPATH
export FPATH=/n/home05/haehn/nolearncox/src/hdf5-1.8.17/hdf5/include:$FPATH

source /n/home05/haehn/nolearncox/bin/activate


cd /n/home05/haehn/Projects/3dxp/
python threed.py {DATAPATH} {Y} {X} {OUTPUTPATH}

# end of program
exit 0;
'''

In [41]:
for Y in tiles_Y:
    for X in tiles_X:

        t2 = t.replace('{MEMORY}', str(memory))
        t2 = t2.replace('{DATAPATH}', str(CLUSTER_DATA))
        t2 = t2.replace('{X}', str(X))
        t2 = t2.replace('{Y}', str(Y))
        t2 = t2.replace('{OUTPUTPATH}', str(CLUSTER_STL_OUTPUT))        
        
        with open('SLURM/mesh-'+str(Y)+'-'+str(X)+'.slurm', 'w') as f:
            f.write(t2)